# Código main para coleta dos tickets e indicadores de valuation

## Código 1 para coleta dos tickets no site da B3

In [2]:
from selenium import webdriver
import time 
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
# Abre o Chrome
wd = webdriver.Chrome()
url = 'https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm'
wd.get(url)
time.sleep(0.3) #Espera a página ser carregada

# Fecha aba de cookies
wd.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]').click()
time.sleep(0.2)

# Muda para o iframe que tem o botão
wd.switch_to.frame('bvmf_iframe')

# Clica no botão 'todos'
wd.find_element(By.XPATH, '//*[@id="accordionName"]/div/app-companies-home-filter-name/form/div/div[4]/button').click()
time.sleep(0.3)

# Clica no botão que transforma em 'tabela'
wd.find_element(By.XPATH, '//*[@id="nav-table-tab"]/i').click()
time.sleep(0.3)

# Clica para aparecer '120' ações na tabela que é o máximo
Select(wd.find_element(By.XPATH, '//*[@id="selectPage"]')).select_by_value('120')
time.sleep(0.3)

# Cria lista vazia que recebe os tickets
tickets = []

# Salva número total de tickets em uma variável
b3_content = BeautifulSoup(wd.page_source, 'html.parser')
qtd_tickets_b3 = int(b3_content.find('div', attrs = {'class' : 'col-lg-11 col-sm-12 col-12 text-right mt-1'})
                               .text.strip()[0:5]
                               .replace('.', ''))

'''Faz um loop de paginação com um click 
   passando para próxima página, coloca um stop
   sendo qualquer erro ou quando o número de tickets
   bate com o site da B3'''
while True:
    try:
        # Clica no botão next da paginação
        wd.find_element(By.CLASS_NAME, 'pagination-next').click()
        time.sleep(0.5)
        
        # Coleta tabela em html
        b3_table = b3_content.find('table', attrs = {'class' : 'table table-responsive-sm table-responsive-md'})
        
        # Converte para pandas e retorna os tickets em uma lista
        df_b3 = pd.read_html(str(b3_table))[0]
#         print(df_b3['Código'].tolist())
        
        # Adiciona os novos tickets na lista
        tickets = tickets + df_b3['Código'].tolist()
#         print(len(tickets))
        if len(tickets) >= qtd_tickets_b3 :
            break
        # Coleta o html do site
        b3_content = BeautifulSoup(wd.page_source, 'html.parser')
    except:
        break

assert len(tickets) == qtd_tickets_b3, 'O total de dados não bate com o total da B3'

# Pega cada ticket e transforma nas ações negociadas
acoes_ordinarias = [] # Final 3
acoes_preferenciais = [] # Final 4 5 6
acoes_units = [] # Final 11

for element in tickets:
    acoes_ordinarias.append(element + '3')
    acoes_preferenciais.append(element + '4')
    acoes_preferenciais.append(element + '5')
    acoes_preferenciais.append(element + '6')
    acoes_units.append(element + '11')
    
total_tickets_b3 = acoes_ordinarias + acoes_preferenciais + acoes_units

assert (5*qtd_tickets_b3) == len(total_tickets_b3), 'O total de açoes(P,O,U) não bate com o total da B3'

## Cria o dataframe final com os dados do status invest (Código 2)

In [4]:
cont_df = 0
wd = webdriver.Chrome()
for acoes in total_tickets_b3:
    try:
        url = 'https://statusinvest.com.br/acoes/' + acoes
        wd.get(url)
        time.sleep(2) #Espera a página ser carregada

        # Seleciona botão de histórico
        wd.find_element(By.XPATH, '//*[@id="indicators-section"]/div[1]/div[2]/button[2]/div').click()
        time.sleep(2)
        
        # Coleta de indicadores
        # Pega o conteudo da página em html
        status_invest = BeautifulSoup(wd.page_source, 'html.parser')

        # Tabelas de indicadores e valores
        tabelas_ind_vlr = status_invest.find_all('div', attrs = {'class' : 'd-flex w-100'})

        ''' Cria lista que recebe os indicadores
            de todas as tabelas'''
        lista_indicadores = []

        # Loop em cada tabela coletando os indicadores
        for n_tab in range(len(tabelas_ind_vlr)):

            indicadores = tabelas_ind_vlr[n_tab]\
                          .find_all('div', attrs = {'class' : 'indicador d-flex justify-between align-items-center'})

            #Coleta cada indicador de cada tabela
            for n_indicadores in range(len(indicadores)):
                lista_indicadores.append(indicadores[n_indicadores]['data-indicador'])
        print(acoes)
#         print(lista_indicadores)

        # Coleta os valores de cada tabela em uma única lista
        lista_valores = []
        for n_tab in range(len(tabelas_ind_vlr)):
            valores_html = tabelas_ind_vlr[n_tab]\
                           .find_all('div', attrs = {'class' : 'td w-100 timeType-0'})
            for element in valores_html: 
                lista_valores.append(element.text)
#         print(lista_valores)

        # Cria tabela de zeros para receber os indicadores e valores
        n_linhas = len(lista_indicadores) # Nome dos indicadores
        
        # Pega as datas das colunas pois essas mudam em cada ticket
        colunas = []
        tabela_0 = tabelas_ind_vlr[0]\
                   .find('div', attrs = {'class' : 'tr w-100 d-flex justify-start'})\
                   .find_all('div', attrs = {'class' : 'th w-100 timeType-0'})
        for year in tabela_0:
            colunas.append(year.text)
            
        n_colunas = len(colunas)
        tabela_final = np.zeros([n_linhas, n_colunas+1], dtype ='object') # +1 para coluna de indicadores
        
        # Preenche o índice com o número dos indicadores
        tabela_final[:,0] = np.array(lista_indicadores, dtype = 'object')
        
        # preenche cada linha por ordem da lista com os valores
        cont = 0
        for i in range (n_linhas) :
            for j in range (1, n_colunas+1) :
                tabela_final[i,j] = lista_valores[cont]
                cont+=1
        
        # Cria o dataframe completo com os valores
        colunas = ['anos']+colunas
        dataset = pd.DataFrame(tabela_final, columns = colunas)
        # Transpoem o dataframe e cria a coluna de data 
        dataset = dataset.transpose().reset_index()
        # Renomeia a primeira linha sendo as colunas
        columns = dataset.iloc[0,:]
        dataset.columns = columns
        # Pega somente os valores necessários, resetando o index para começar com 0
        dataset = dataset.iloc[1:].reset_index(drop = True)
        # Cria a coluna com o ticket em uma posição específica
        dataset.insert (1, "ticket", acoes)
        
        if cont_df == 0:
            dataset_final = dataset.copy()
            cont_df += 1
        else:
            dataset_final = pd.concat([dataset_final, dataset])
    except:
        pass

RRRP3
AERI3
AESB3
AFLT3
AGXY3
RPAD3
ALSO3
ALLD3
ALPK3
ALPA3
APER3
AVLL3
ALUP3
ABEV3
AMBP3
AMER3
CBEE3
ANIM3
ARZZ3
ARML3
CRFB3
ATMP3
ATOM3
AURE3
AZEV3
B3SA3
BAHI3
MODL3
BEES3
BDLL3
BALM3
BBSE3
BBML3
BRIV3
BAZA3
BBDC3
BBAS3
BPAC3
BGIP3
BPAR3
BRSR3
BMIN3
BMEB3
BNBR3
PINE3
SANB3
BMOB3
BMKS3
BIOM3
BLAU3
BLUT3
SOJA3
BOAS3
BOBR3
BRBI3
BRML3
BRPR3
BRAP3
AGRO3
BRKM3
BSLI3
BRFS3
BRIT3
BRQB3
CXSE3
CAMB3
CAML3
CCRO3
CEAB3
MAPT3
ELET3
CLSC3
AALR3
PCAR3
CASN3
GPAR3
CEGR3
CEEB3
CEBR3
CMIG3
CEPE3
COCE3
CSRN3
CEED3
FESA3
CEDO3
CGAS3
CATA3
MSPA3
CPLE3
PEAB3
SBSP3
CSMG3
SAPR3
CSAB3
CSNA3
CTNM3
CTSA3
CIEL3
CMSA3
CNSY3
CLSA3
VVEO3
COGN3
CBAV3
BRGE3
CALI3
TEND3
CORR3
CSAN3
CPFE3
CPRE3
CRDE3
MERC3
CSED3
CMIN3
CSUD3
CTCA3
TRPL3
CURY3
CVCB3
CYRE3
DMVF3
DESK3
DXCO3
DEXP3
DASA3
PNVL3
DIRR3
DOHL3
DMMO3
DOTZ3
DTCY3
ECOR3
ENBR3
EALT3
EKTR3
LIPR3
ELMD3
ELPL3
EMAE3
EPAR3
EMBR3
PGMN3
ENAT3
ENMT3
ENGI3
ENEV3
EGIE3
ENJU3
EQTL3
EQPA3
ETER3
EUCA3
EVEN3
EZTC3
VSPT3
FHER3
CRIV3
FLRY3
FLEX3
FRAS3
GFSA3
GSHP3
GGBR3
GETT3
NINJ

In [5]:
dataset_final.head()

,anos,ticket,dy,p_l,peg_Ratio,p_vp,ev_ebitda,ev_ebit,p_ebita,p_ebit,vpa,p_ativo,lpa,p_sr,p_capitlgiro,p_ativocirculante,dividaliquida_patrimonioliquido,dividaliquida_ebitda,dividaliquida_ebit,patrimonio_ativo,passivo_ativo,liquidezcorrente,margembruta,margemebitda,margemebit,margemliquida,roe,roa,roic,giro_ativos,receitas_cagr5,lucros_cagr5
0,2022,RRRP3,-%,"35,28","-0,17","1,78",-,"14,64",-,"15,20","21,27","0,95","1,07","5,02","8,93","-1,23","-0,07",-,"-0,56","0,54","0,46","1,88","55,99%",-%,"33,03%","14,23%","5,03%","2,70%","6,22%","0,19",-%,-%
1,2021,RRRP3,-%,"-7.510,49",-,"1,65",-,"10,45",-,"16,38","20,28","1,25","0,00","9,34","3,17","-2,48","-0,60",-,"-5,93","0,75","0,23","4,79","56,10%",-%,"57,03%","-0,12%","-0,02%","-0,02%","8,39%","0,13",-%,-%
2,2020,RRRP3,-%,"-15,71",-,"3,78",-,"-23,21",-,"-23,64","9,84","1,93","-2,37","19,94","6,20","-3,05","-0,07",-,"0,42","0,51","0,49","6,74","47,99%",-%,"-84,38%","-126,92%","-24,07%","-12,29%","-11,55%","0,10",-%,-%
3,2019,RRRP3,-%,-,-,-,-,-,-,-,-,-,-,-,-,-,"-0,27",-,"4,16","0,60","0,40","2,71","50,52%",-%,"-47,16%","-98,66%","-13,50%","-8,06%","-8,07%","0,08",-%,-%
4,2018,RRRP3,-%,-,-,-,-,-,-,-,-,-,-,-,-,-,"-0,32",-,"-3,25","0,60","0,40","3,14","65,45%",-%,"75,48%","-34,93%","-4,60%","-2,76%","8,57%","0,08",-%,-%


In [7]:
# Número de tickets
len(dataset_final['ticket'].unique().tolist())

537

In [9]:
dataset_final.to_csv('statusinvest.csv', index = False) 

## Check the tickets with IBOV and IBRX100

In [11]:
# Coleta tickets IBOV
wd = webdriver.Chrome()
url = '''https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm'''
wd.get(url)
time.sleep(0.3) #Espera a página ser carregada
# Fecha aba de cookies
wd.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]').click()
time.sleep(0.2)

# Muda para o iframe que tem o botão
wd.switch_to.frame('bvmf_iframe')
# Seleciona os 120 para filtro
# Clica para aparecer '120' ações na tabela que é o máximo
Select(wd.find_element(By.XPATH, '//*[@id="selectPage"]')).select_by_visible_text('120')
time.sleep(0.2)
# get table
table_html = wd.find_element(By.XPATH, '//*[@id="divContainerIframeB3"]/div/div[1]/form/div[2]/div/table').get_attribute('outerHTML')
table_ibov_tickets = pd.read_html(table_html)[0]
tickets_ibov = table_ibov_tickets['Código'].tolist()[:-2]
print(len(tickets_ibov))
# Verifica o primeiro item e o último da lista
print(tickets_ibov[0])
print(tickets_ibov[-1])

89
RRRP3
YDUQ3


In [12]:
# Coleta tickets IBRX100
wd = webdriver.Chrome()
url = '''https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-brasil-100-ibrx-100-composicao-da-carteira.htm'''
wd.get(url)
 #Espera a página ser carregada
time.sleep(0.3)
# Fecha aba de cookies
wd.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]').click()
time.sleep(0.2)
# Muda para o iframe que tem o botão
wd.switch_to.frame('bvmf_iframe')
# Seleciona os 120 para filtro
# Clica para aparecer '120' ações na tabela que é o máximo
Select(wd.find_element(By.XPATH, '//*[@id="selectPage"]')).select_by_visible_text('120')
time.sleep(0.2)
# get table
table_html = wd.find_element(By.XPATH, '//*[@id="divContainerIframeB3"]/div/div[1]/form/div[2]/div/table').get_attribute('outerHTML')
table_ibrx100_tickets = pd.read_html(table_html)[0]
tickets_ibrx100 = table_ibrx100_tickets['Código'].tolist()[:-2]
print(len(tickets_ibrx100))
# Verifica o primeiro item e o último da lista
print(tickets_ibrx100[0])
print(tickets_ibrx100[-1])

100
RRRP3
YDUQ3


In [13]:
# Pega os tickets do webscraping
tickets_stinvest = dataset_final['ticket'].unique().tolist() 

In [14]:
# Tickets que estão no IBOV e não no webscraping
list(set(tickets_ibov) - set(tickets_stinvest))

[]

In [15]:
list(set(tickets_ibrx100) - set(tickets_stinvest))

[]

# Web scraping dos BDRS

In [48]:
from selenium import webdriver
import time 
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

wd = webdriver.Chrome()
url = 'https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/bdrs/bdrs-nao-patrocinados/bdrs-nao-patrocinados-listados/'
wd.get(url)
time.sleep(1) #Espera a página ser carregada

# Fecha aba de cookies
wd.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]').click()
time.sleep(0.2)

# Muda para o iframe que tem o botão
wd.switch_to.frame('bvmf_iframe')

# Clica no botão 'todos'
wd.find_element(By.XPATH, '//*[@id="divContainerIframeB3"]/div/div/app-companies-home-filter-bdrs/form/div/div/div/div[3]/button').click()
time.sleep(0.5)

# Clica no botão que transforma em 'tabela'
wd.find_element(By.XPATH, '//*[@id="nav-table-tab"]/i').click()
time.sleep(0.5)

# Clica para aparecer '120' ações na tabela que é o máximo
Select(wd.find_element(By.XPATH, '//*[@id="selectPage"]')).select_by_value('120')
time.sleep(0.5)

# Cria lista vazia que recebe os tickets
tickets_bdrs = []

# Salva número total de tickets em uma variável
b3_content = BeautifulSoup(wd.page_source, 'html.parser')
qtd_bdrs_b3 = int(b3_content.find('div', attrs = {'class' : 'col-lg-11 col-sm-12 col-12 text-right mt-1'})
                            .text.strip()[0:3]
                            .replace('.', ''))


'''Faz um loop de paginação com um click 
   passando para próxima página, coloca um stop
   sendo qualquer erro ou quando o número de tickets
   bate com o site da B3'''
while True:
    try:
        # Clica no botão next da paginação
        wd.find_element(By.CLASS_NAME, 'pagination-next').click()
        time.sleep(0.8)
        
        # Coleta tabela em html
        b3_table = b3_content.find('table', attrs = {'class' : 'table table-responsive-sm table-responsive-md'})
        
        # Converte para pandas e retorna os tickets em uma lista
        df_b3 = pd.read_html(str(b3_table))[0]
#         print(df_b3['Código'].tolist())
        
        # Adiciona os novos tickets na lista
        tickets_bdrs = tickets_bdrs + df_b3['Código'].tolist()
#         print(len(tickets))
        if len(tickets_bdrs) >= qtd_bdrs_b3 :
            break
        # Coleta o html do site
        b3_content = BeautifulSoup(wd.page_source, 'html.parser')
    except:
        break

assert len(tickets_bdrs) == qtd_bdrs_b3, 'O total de dados não bate com o total da B3'

# Pega cada ticket e transforma nas ações negociadas
bdr_nao_patrocinados = [] # Final 34 e 35
bdr_patrocinados = ['AURA32', 'G2DI32', 'GPIV32', 'INBR32', 'NUBR32', 'PPLA32', 'STOC32', 'XPBR32',
                    'AURA33', 'G2DI33', 'GPIV33', 'INBR33', 'NUBR33', 'PPLA33', 'STOC33', 'XPBR33',
                    'XPBR31'] # Final 32 e 33

for element in tickets_bdrs:
    bdr_nao_patrocinados.append(element + '34')
    bdr_nao_patrocinados.append(element + '35')

total_bdrs_b3 = bdr_nao_patrocinados + bdr_patrocinados

assert (2*qtd_bdrs_b3+17) == len(total_bdrs_b3), 'O total de bdrs não bate com o total da B3'

## Coleta do dataframe

In [49]:
cont_df = 0
wd = webdriver.Chrome()
for bdrs in total_bdrs_b3:
    try:
        url = 'https://statusinvest.com.br/bdrs/' + bdrs
        wd.get(url)
        time.sleep(2.5) #Espera a página ser carregada
        # Seleciona botão de histórico
        wd.find_element(By.XPATH, '//*[@id="indicators-section"]/div[1]/div[2]/button[2]/div').click()
        time.sleep(0.5)
        # Coleta de indicadores
        # Pega o conteudo da página em html
        status_invest = BeautifulSoup(wd.page_source, 'html.parser')
        # Tabelas de indicadores e valores
        tabelas_ind_vlr = status_invest.find_all('div', attrs = {'class' : 'd-flex w-100'})
        ''' Cria lista que recebe os indicadores
            de todas as tabelas'''
        lista_indicadores = []
        # Loop em cada tabela coletando os indicadores
        for n_tab in range(len(tabelas_ind_vlr)):
            indicadores = tabelas_ind_vlr[n_tab]\
                          .find_all('div', attrs = {'class' : 'indicador d-flex justify-between align-items-center'})
            #Coleta cada indicador de cada tabela
            for n_indicadores in range(len(indicadores)):
                lista_indicadores.append(indicadores[n_indicadores]['data-indicador'])
        print(bdrs)
        # print(lista_indicadores)
        # Coleta os valores de cada tabela em uma única lista
        lista_valores = []
        for n_tab in range(len(tabelas_ind_vlr)):
            linhas_table = tabelas_ind_vlr[n_tab]\
                                   .find_all('div', attrs = {'class' : 'tr w-100 d-flex justify-start asset-last'})
            for linha in linhas_table:
                valores_html_defined = linha\
                                       .find_all('div', attrs = {'class' : 'td w-100 timeType-0'})
                valores_html_undefined = linha\
                                         .find_all('div', attrs = {'class' : 'td w-100 timeType-undefined'})
                if len(valores_html_defined) != 0:
                    for element in valores_html_defined: 
                        lista_valores.append(element.text)
                if len(valores_html_undefined) !=0:
                        for element in valores_html_undefined: 
                            lista_valores.append(element.text)
        # print(lista_valores)
         # Cria tabela de zeros para receber os indicadores e valores
        n_linhas = len(lista_indicadores) # Nome dos indicadores
         # Pega as datas das colunas pois essas mudam em cada ticket
        colunas = []
        tabela_0 = tabelas_ind_vlr[0]\
                   .find('div', attrs = {'class' : 'tr w-100 d-flex justify-start'})\
                   .find_all('div', attrs = {'class' : 'th w-100 timeType-0'})
        for year in tabela_0:
            colunas.append(year.text)
        n_colunas = len(colunas)
        tabela_final = np.zeros([n_linhas, n_colunas+1], dtype ='object') # +1 para coluna de indicadores
        # Preenche o índice com o número dos indicadores
        tabela_final[:,0] = np.array(lista_indicadores, dtype = 'object')
        # preenche cada linha por ordem da lista com os valores
        cont = 0
        for i in range (n_linhas) :
            for j in range (1, n_colunas+1) :
                tabela_final[i,j] = lista_valores[cont]
                cont+=1
#                 print(i,j)
        # Cria o dataframe completo com os valores
        colunas = ['anos']+colunas
        dataset = pd.DataFrame(tabela_final, columns = colunas)
        # Transpoem o dataframe e cria a coluna de data 
        dataset = dataset.transpose().reset_index()
        # Renomeia a primeira linha sendo as colunas
        columns = dataset.iloc[0,:]
        dataset.columns = columns
        # Pega somente os valores necessários, resetando o index para começar com 0
        dataset = dataset.iloc[1:].reset_index(drop = True)
        # Cria a coluna com o ticket em uma posição específica
        dataset.insert (1, "ticket", bdrs)   
        if cont_df == 0:
            dataset_final = dataset.copy()
            cont_df += 1
            print('ok')
        else:
            dataset_final = pd.concat([dataset_final, dataset])
            print('ok')
    except:
        pass

MMMC34
ok
A1BB34
ok
ABTT34
ok
ABBV34
ok
ABAM34
ok
A2SO34
ok
ACNB34
ok
ATVI34
ok
ADBE34
ok
A1AP34
ok
A1EG34
ok
A1GI34
ok
A1PD34
ok
AIRB34
ok
A1KA34
ok
AKZA34
ok
A1LK34
ok
A1LB34
ok
A2LC34
ok
A1RE34
ok
BABA34
ok
A1LG34
ok
A1GN34
ok
ALFG34
ok
A1EN34
ok
A1TT34
ok
A1LN34
ok
GOGL34
ok
GOGL35
ok
A1YX34
ok
MOOO34
ok
AMZO34
ok
A2MB34
ok
A2MC34
ok
A1CR34
ok
D2OX34
ok
A1EE34
ok
A1MX34
ok
AALL34
ok
A1EP34
ok
AXPB34
ok
AIGB34
ok
T1OW34
ok
A1WK34
ok
A1MP34
ok
A1MB34
ok
A1ME34
ok
AMGN34
ok
A1PH34
ok
A2MR34
ok
A1DI34
ok
AAGO34
A1UA34
ok
ABUD34
ok
N2LY34
ok
A1NS34
ok
A1OS34
ok
A1ON34
ok
A1PA34
ok
A1IV34
ok
AAPL34
ok
A1MT34
ok
APTV34
ok
ARMT34
ok
A1DM34
ok
A1RG34
ok
A1NE34
ok
AWII34
ok
A2RW34
ok
A2RR34
ok
A1SN34
ok
ASML34
ok
A1SU34
ok
A1ZN34
ok
ATTB34
ok
T1AM34
ok
A1TM34
ok
A1UT34
ok
A1TH34
ok
ADPR34
ok
AZOI34
ok
A1VB34
ok
A1VY34
ok
A2XO34
ok
A2ZT34
ok
BIDU34
ok
B1KR34
ok
B1LL34
ok
BILB34
ok
B1SA34
ok
BCSA34
ok
C2OL34
ok
BOAC34
ok
B1CS34
ok
BARR34
ok
B1AX34
ok
B1DX34
ok
B1GN34
ok
BERK34
ok
BBYY34
ok
B2Y

In [50]:
dataset_final.to_csv('bdrs_stinvest.csv', index = False) 

# Web scraping FII

In [2]:
from selenium import webdriver
import time 
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

wd = webdriver.Chrome()
url = 'https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/fundos-de-investimentos/fii/fiis-listados/'
wd.get(url)
time.sleep(3) #Espera a página ser carregada

# Fecha aba de cookies
wd.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]').click()
time.sleep(1)

# Muda para o iframe que tem o botão
wd.switch_to.frame('bvmf_iframe')

# Clica no botão 'todos'
wd.find_element(By.XPATH, '//*[@id="divContainerIframeB3"]/form/div[1]/div/div/div[1]/div/div[3]/button').click()
time.sleep(1)

# Clica no botão que transforma em 'tabela'
wd.find_element(By.XPATH, '//*[@id="nav-table-tab"]/i').click()
time.sleep(1)

# Clica para aparecer '120' ações na tabela que é o máximo
Select(wd.find_element(By.XPATH, '//*[@id="selectPage"]')).select_by_visible_text('60')
time.sleep(1)

# Cria lista vazia que recebe os tickets
tickets_fii = []

# Salva número total de tickets em uma variável
b3_content = BeautifulSoup(wd.page_source, 'html.parser')
qtd_fii_b3 = int(b3_content.find('p').find('strong').text)


'''Faz um loop de paginação com um click 
   passando para próxima página, coloca um stop
   sendo qualquer erro ou quando o número de tickets
   bate com o site da B3'''
while True:
    try:

        # Coleta tabela em html
        b3_table = b3_content.find('table', attrs = {'class' : 'table table-responsive-sm table-responsive-md'})
#         print('aqui2')

        # Converte para pandas e retorna os tickets em uma lista
        df_b3 = pd.read_html(str(b3_table))[0]
#         print(df_b3['Código'].tolist())
        
        # Adiciona os novos tickets na lista
        tickets_fii = tickets_fii + df_b3['Código'].tolist()
#         print(len(tickets_fii))

        if len(tickets_fii) >= qtd_fii_b3 :
#             print('parou')
            break
            
        # Clica no botão next da paginação
        wd.find_element(By.CLASS_NAME, 'pagination-next').click()
        time.sleep(2)
        
        # Coleta o html do site
        b3_content = BeautifulSoup(wd.page_source, 'html.parser')
#         print('final antes do break')
        
    except:
        break

assert len(tickets_fii) == qtd_fii_b3, 'O total de dados não bate com o total da B3'

# Pega cada ticket e transforma nas ações negociadas
total_fii_b3 = [] # Final 11

for element in tickets_fii:
    total_fii_b3.append(element + '11')

assert qtd_fii_b3 == len(total_fii_b3), 'O total de FII não bate com o total da B3'

In [3]:
## Coleta os dados e transforma em um dataframe
cont_df = 0
wd = webdriver.Chrome()
for ticket in total_fii_b3:
    try:
        url = 'https://statusinvest.com.br/fundos-imobiliarios/' + ticket
        wd.get(url)
        time.sleep(3) #Espera a página ser carregada
        #Clica no botão 'ver mais'
        wd.find_element(By.XPATH, '//*[@id="contabil-section"]/div[1]/div/div[2]/div[2]/button').click()
        time.sleep(1.2)
        #Clica no botão 'anual'
        wd.find_element(By.XPATH, '//*[@id="contabil-section"]/div[1]/div/div[2]/header/div[2]/div/div/label/span[1]').click()
        time.sleep(4)
        # Read table
        dataset = pd.read_html(wd.page_source)[1]
        #Exclude variation columns
        dataset = dataset[[col for col in dataset.columns if col.startswith("AH") == False]]
        # Transpose
        dataset = dataset.transpose().reset_index()
        # Renomeia a primeira linha sendo as colunas
        columns = dataset.iloc[0, :]
        dataset.columns = columns
        dataset = dataset.rename(columns={"#": "Anos"})
        # Pega somente os valores necessários, resetando o index para começar com 0
        dataset = dataset.iloc[1:].reset_index(drop = True)
        # Cria a coluna com o ticket em uma posição específica
        dataset.insert (1, "Ticket", ticket)   
        if cont_df == 0:
            dataset_final = dataset.copy()
            cont_df += 1
            print(ticket)
            print('ok')
        else:
            dataset_final = pd.concat([dataset_final, dataset])
            print(ticket)
            print('ok')
    except:
        pass
    
dataset_final.to_csv('FII_stinvest.csv', index = False) 

CCME11
ok
ITIT11
ok
JASC11
ok
LAVF11
ok
AFHI11
ok
ALZM11
ok
MTOF11
ok
ALZR11
ok
AURB11
ok
ASMT11
ok
AIEC11
ok
BCRI11
ok
BNFS11
ok
BZEL11
ok
BPLC11
ok
BBFO11
ok
BBPO11
ok
BBIM11
ok
BBRC11
ok
RDPD11
ok
RNDP11
ok
BLCA11
ok
BLMC11
ok
BLMG11
ok
BLMO11
ok
BLMR11
ok
BCIA11
ok
BREV11
ok
BZLI11
ok
CARE11
ok
BRCO11
ok
BICE11
ok
BIME11
ok
BRIM11
ok
BRIP11
ok
BIPD11
ok
BRIX11
ok
LLAO11
ok
BTLG11
ok
BTWR11
ok
BTSG11
ok
BTSI11
ok
CRFF11
ok
CXRI11
ok
CPFF11
ok
CPTS11
ok
CACR11
ok
CBOP11
ok
CFHI11
ok
CFII11
ok
CJCT11
ok
HGFF11
ok
HGLG11
ok
HGPO11
ok
HGRE11
ok
HGCR11
ok
HGRU11
ok
HGRS11
ok
CVPR11
ok
CYCR11
ok
CYLD11
ok
DLMT11
ok
DPRO11
ok
DEVA11
ok
EGYR11
ok
EQIR11
ok
ERCR11
ok
ERPA11
ok
KEVE11
ok
KINP11
ok
EXES11
ok
FLCR11
ok
VRTA11
ok
BMII11
ok
LRDI11
ok
MCHY11
ok
MMPD11
ok
IBCR11
ok
MGCR11
ok
GAME11
ok
FAED11
ok
BMLC11
ok
BPRP11
ok
BRCR11
ok
BCFF11
ok
FCFL11
ok
CNES11
ok
CEOC11
ok
EDGA11
ok
HCRI11
ok
NSLU11
ok
HTMX11
ok
MAXR11
ok
NCHB11
ok
NVHO11
ok
PQDP11
ok
PATB11
ok
RBRM11
ok
RBRR11
ok
RECR11
ok


In [20]:
df.to_csv('FII_stinvest.csv', index = False)